In [1]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
import datetime
import os
import configparser
import sys
from rechunker import rechunk
import zarr 
from datetime import datetime
import json

In [2]:
print("importing functions ...")
# Append the directory of the module to sys.path - import functions
sys.path.append('/g/data/es60/users/thomas_moore/code/Climatology-generator-demo/src/')
import bran2020_demo_functions as my_tools
from bran2020_demo_functions import keep_only_selected_vars, load_stats_config, print_chunks, stats_monthclim, median_monthclim, quantile_monthclim
# load config
config = load_stats_config()
print(">>> config: "+str(config))
var = config['variable']
print("variable requested: "+var)
zarr_path_dict = config['zarr_path_dict']
print(">>> zarr_path_dict: "+str(zarr_path_dict))
write_results_base_dir = config['write_results_base_dir']
print(">>> write_results_base_dir: "+str(write_results_base_dir))
n_workers = config['n_workers']
print(">>> n_workers: "+str(n_workers))
threads_per_worker = config['threads_per_worker']
print(">>> threads_per_worker: "+str(threads_per_worker))
memory_limit = config['memory_limit']
print(">>> memory_limit: "+str(memory_limit))
run_base_stats = config['run_base_stats']
print(">>> run_base_stats: "+str(run_base_stats))
run_quant = config['run_quant']
print(">>> run_quant: "+str(run_quant))
run_all_time = config['run_all_time']
print(">>> run_all_time: "+str(run_all_time))
run_neutral = config['run_neutral']
print(">>> run_neutral: "+str(run_neutral))
run_la_nina = config['run_la_nina']
print(">>> run_la_nina: "+str(run_la_nina))
run_el_nino = config['run_el_nino']
print(">>> run_el_nino: "+str(run_el_nino))
lat_name_dict = config['lat_name_dict']
print(">>> lat_name_dict: "+str(lat_name_dict))
lon_name_dict = config['lon_name_dict']
print(">>> lon_name_dict: "+str(lon_name_dict))
time_name = config['time_name']
print(">>> time_name: "+str(time_name))
# ------
print(">>> Spinning up a dask cluster...")

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=28,threads_per_worker=1)
client = Client(cluster)
print(client)
# Get current date and time
now = datetime.now()
# Format as a string
timestamp_str = now.strftime("%Y.%m.%d.%H.%M.%S")
print('timestamp: '+timestamp_str)

importing functions ...
>>> config: {'variable': 'temp', 'zarr_path_dict': {'temp': '/scratch/es60/ard/reanalysis/BRAN2020/ARD/looped_rechunk_output/temp/temp_combined_output.zarr', 'salt': '/scratch/es60/ard/reanalysis/BRAN2020/ARD/looped_rechunk_output/salt/salt_combined_output.zarr', 'u': '/scratch/es60/ard/reanalysis/BRAN2020/ARD/looped_rechunk_output/u/u_combined_output.zarr', 'v': '/scratch/es60/ard/reanalysis/BRAN2020/ARD/looped_rechunk_output/v/v_combined_output.zarr', 'mld': 'None', 'eta_t': 'None'}, 'write_results_base_dir': '/g/data/es60/users/thomas_moore/clim_demo_results/daily/draft_delivery/', 'n_workers': 48, 'threads_per_worker': 1, 'memory_limit': '60GB', 'run_base_stats': True, 'run_quant': True, 'run_all_time': True, 'run_neutral': False, 'run_la_nina': False, 'run_el_nino': False, 'lat_name_dict': {'temp': 'yt_ocean', 'salt': 'yt_ocean', 'u': 'yu_ocean', 'v': 'yu_ocean', 'mld': 'yt_ocean', 'eta_t': 'yt_ocean'}, 'lon_name_dict': {'temp': 'xt_ocean', 'salt': 'xt_ocea

In [3]:
print(">>> running base stats ...")
#
lat_name = lat_name_dict[var]
lon_name = lon_name_dict[var]
vars_to_keep=[var,time_name,'st_ocean',lat_name,lon_name]
xarray_open_kwargs = {'chunks': {time_name: -1,'st_ocean':10}}

>>> running base stats ...


In [4]:
lat_name

'yt_ocean'

In [5]:
lon_name

'xt_ocean'

In [6]:
vars_to_keep

['temp', 'Time', 'st_ocean', 'yt_ocean', 'xt_ocean']

In [7]:
xarray_open_kwargs

{'chunks': {'Time': -1, 'st_ocean': 10}}

In [10]:
ds = xr.open_mfdataset('/g/data/gb6/BRAN/BRAN2020/daily/ocean_'+var+'_*.nc',
                        parallel=True,chunks=xarray_open_kwargs['chunks'],
                        preprocess=lambda ds: keep_only_selected_vars(ds, vars_to_keep=vars_to_keep))
print_chunks(ds[var])

Time chunks: (31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31

{'Time': (31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  29,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  29,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  29,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  31,
  30,
  31,
  31,
  28,
  31,
  30,
  31,
  30,
  31,
  31,
  30,
  

In [9]:
xarray_open_kwargs['chunks']

{'Time': -1, 'st_ocean': 10}

In [16]:
stats_monthclim_ds = stats_monthclim(ds,var_name=var,time_dim=time_name,skipna_flag=False,method_str='cohorts')
print(stats_monthclim_ds.nbytes/1e9)
# write to netcdf
results_path = write_results_base_dir
results_file = 'BRAN2020_base_stats_'+var+'_'+timestamp_str+'.nc'

print("writing to the base stats netcdf file for : "+var+" ....")

# Specify chunks
chunks = {'month':12,'st_ocean': 1, lat_name:1500, lon_name:3600}

# Specify encoding
encoding = {}
for var_name in stats_monthclim_ds.data_vars:
    encoding[var_name] = {'zlib': True, 'complevel': 5, 'dtype': 'float32'}

# Save to NetCDF with chunking and encoding
stats_monthclim_ds.chunk(chunks)

52.876841304
writing to the base stats netcdf file for : temp ....


<xarray.Dataset> Size: 53GB
Dimensions:    (month: 12, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * xt_ocean   (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean   (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.85 74.95
  * st_ocean   (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    mean_temp  (month, st_ocean, yt_ocean, xt_ocean) float32 13GB dask.array<chunksize=(12, 1, 1500, 3600), meta=np.ndarray>
    std_temp   (month, st_ocean, yt_ocean, xt_ocean) float32 13GB dask.array<chunksize=(12, 1, 1500, 3600), meta=np.ndarray>
    max_temp   (month, st_ocean, yt_ocean, xt_ocean) float32 13GB dask.array<chunksize=(12, 1, 1500, 3600), meta=np.ndarray>
    min_temp   (month, st_ocean, yt_ocean, xt_ocean) float32 13GB dask.array<chunksize=(12, 1, 1500, 3600), meta=np.ndarray>
Attributes:
    filename:           TMP/ocean_ofam_1993_01_01.nc.0000
    NumFilesInSet:      20
    grid_type:          regular
    grid_tile:          N/A
    history:            Mon May 18 20:07:58 2020: ncrcat -4 --dfl_lvl 1 --cnk...
    NCO:                netCDF Operators version 4.9.2 (Homepage = http://nco...
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...
    title:              BRAN2020

In [12]:
chunks

{'mean_temp': {'st_ocean': 1},
 'std_temp': {'st_ocean': 1},
 'max_temp': {'st_ocean': 1},
 'min_temp': {'st_ocean': 1}}